<a href="https://colab.research.google.com/github/pepperamy/disclosure_change/blob/main/change_siamese_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# initialization

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
import sys, os

use_colab = False
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    
    !pip install linearmodels
    
    cur_path = os.path.join('/content/drive/MyDrive/change')
    os.chdir(cur_path)

    !pip install transformers

    !pip install -U sentence-transformers

    !pip install torchinfo

    !pip install tqdm

    
    !pip install numba
    
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
      print('and then re-execute this cell.')
    else:
      print(gpu_info)


else:
    cur_path = os.path.join('H:\\My Drive\\change')
    os.chdir(cur_path)
    import torch
    torch.cuda.is_available()



C:\Users\KK\Anaconda3\envs\py38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\KK\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\KK\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


True

In [ ]:
import random, pickle
import numpy as np
from torch.nn import BCEWithLogitsLoss, BCELoss, MSELoss
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support
import tensorflow as tf
import torch
import pandas as pd

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForSequenceClassification

#from sentence_transformers import SentenceTransformer, util

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torchinfo import summary
from sklearn.model_selection import train_test_split, StratifiedKFold

import copy
from sklearn.utils import shuffle
import glob

import time
import datetime

from itertools import cycle

from tqdm import tqdm

from numba import cuda 

In [ ]:
# # Get the GPU device name.
# device_name = torch.cuda.get_device_name()
# print(device_name)
# # The device name should look like the following:
# if device_name == '/device:GPU:0':
#     print('Found GPU at: {}'.format(device_name))
# else:
#     device_name ="/cpu:0"
#     print('GPU device not found')
#     #raise SystemError('GPU device not found')

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1060 6GB


In [ ]:
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved
# print(f// 1024 ** 2)
from pynvml import *
def get_free_gpu():
    nvmlInit()
    h = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(h)
    print(f'total    : {info.total // 1024 ** 2}')
    print(f'free     : {info.free// 1024 ** 2}')
    print(f'used     : {info.used// 1024 ** 2}')
    #print(torch.cuda.memory_summary()) 


In [ ]:
get_free_gpu()

total    : 6144
free     : 4576
used     : 1567


In [ ]:
# def get_free_gpu():
#     t = torch.cuda.get_device_properties(0).total_memory
#     r = torch.cuda.memory_reserved(0)
#     a = torch.cuda.memory_allocated(0)
#     f = r-a  # free inside reserved
#     print(f// 1024 ** 2)
    

In [ ]:
# from pynvml import *
# nvmlInit()
# h = nvmlDeviceGetHandleByIndex(0)
# info = nvmlDeviceGetMemoryInfo(h)
# print(f'total    : {info.total}')
# print(f'free     : {info.free}')
# print(f'used     : {info.used}')

# load data

In [ ]:
#para_map = pickle.load(open("./data/paragraphs_1994_2016-001.pkl", 'rb'))
#para_map = pickle.load(open("data/mda/paragraphs_1994_2016_original.pkl", 'rb'))
# len(para_map)

In [ ]:
#cik_list = list(para_map.keys())

In [ ]:
# para_test only contian 20 1750
para_map = pickle.load(open("./data/para_test.pkl","rb"))

In [ ]:
pos_neg_pair = pd.read_csv('./data/pos_neg_pair.csv')

In [ ]:
pos_neg_pair.head()

,cik,fyear,count_para,fraud,fyear_bf
0,20,1995,41,0.0,NaN
1,20,1996,31,0.0,1995.0
2,20,1997,34,0.0,1996.0
3,20,1998,49,0.0,1997.0
4,20,1999,40,0.0,1998.0


In [ ]:
pos_neg_pair.tail()

,cik,fyear,count_para,fraud,fyear_bf
151165,1688941,2016,126,0.0,NaN
151166,1689490,2016,55,0.0,NaN
151167,1690666,2016,109,0.0,NaN
151168,1691430,2016,46,0.0,NaN
151169,1695098,2016,27,0.0,NaN


In [ ]:
pos_neg_pair = pos_neg_pair.dropna()

In [ ]:
pos_neg_pair.fraud.value_counts()

0.0    129440
1.0       837
Name: fraud, dtype: int64

In [ ]:
pos_neg_pair

,cik,fyear,count_para,fraud,fyear_bf
1,20,1996,31,0.0,1995.0
2,20,1997,34,0.0,1996.0
3,20,1998,49,0.0,1997.0
4,20,1999,40,0.0,1998.0
5,20,2000,34,0.0,1999.0
...,...,...,...,...,...
151039,1658432,2016,25,0.0,2015.0
151051,1660156,2016,11,0.0,2015.0
151056,1660719,2016,32,0.0,2015.0
151067,1662684,2016,39,0.0,2015.0


In [ ]:
# def get_df_for_cik(cik):
    
#     df = pd.concat({k: pd.Series(v) for k, v in para_map[cik].items()})
#     df = df.reset_index()
    
#     df.columns = ['fyear','pid','text']
#     df = df.sort_values(by=['fyear','pid'])
#     df = df.reset_index(drop = True)
#     df['cik'] = cik
#     df = df[['cik', 'fyear','pid','text']]
    
#     return df


# funtions

## embedding

In [ ]:
# Load the BERT tokenizer.

print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


bert_model = AutoModel.from_pretrained(
    # 'ProsusAI/finbert',
    'bert-base-uncased',
    # 'yiyanghkust/finbert-pretrain',
    num_labels = 2, 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
    )
bert_model.cuda()

Loading BERT tokenizer...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# Put everything together as a function. This is for pretrained word vectors

def get_pretrained_wordvector(sentences, tokenizer, bert_model, max_len=100):

    input_ids = []
    attention_masks = []
    max_len = max_len

    # For every sentence...
    for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        #padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
    
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])


    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    bert_model.eval()
    with torch.no_grad():
        outputs = bert_model(input_ids.to(device), attention_masks.to(device))   
        hidden_states = outputs[2]

    
    # get the last four layers
    token_embeddings = torch.stack(hidden_states[-4:], dim=0) 
    #print(token_embeddings.size())

    # permute axis
    token_embeddings = token_embeddings.permute(1,2,0,3)
    #print(token_embeddings.size())

    # take the mean of the last 4 layers
    token_embeddings = token_embeddings.mean(axis=2)

    #print(token_embeddings.size())
    input_ids.detach().to('cpu')
    attention_masks.detach().to('cpu')
    token_embeddings.detach().to('cpu')
    del input_ids
    return token_embeddings, attention_masks

In [ ]:
def get_text_embedding(cik, fyear, fyear_bf, tokenizer, bert_model, num_para, wrd_len=100, para_map = para_map):
    print(cik, fyear, fyear_bf)
    df = pd.concat({k: pd.Series(v) for k, v in para_map[cik].items()})
    df = df.reset_index()
    df.columns = ['fyear','pid','text']

    input = df[df.fyear == fyear].text.values
    input_bf = df[df.fyear == fyear_bf].text.values

    #get embedding for input
    token_embeddings, masks = get_pretrained_wordvector(input, tokenizer, bert_model, max_len = wrd_len)
    token_embeddings = token_embeddings.to(device) * masks.unsqueeze(-1).to(device) # (atc_num_para, #wrd_len, #dim)
    # padding paragraphs
    print('1 token_embeddings',token_embeddings.size())
    pad_num = num_para - token_embeddings.size()[0]
    if pad_num>0:
        token_embeddings = F.pad(input=token_embeddings, pad=(0,0,0,0,0,pad_num))
        print('2 token_embeddings',token_embeddings.size())
    elif pad_num<0:
        token_embeddings = token_embeddings[0:60]
        print('2 token_embeddings',token_embeddings.size())
    else:
        token_embeddings = token_embeddings

    #get embedding for input_bf
    token_embeddings_bf, masks_bf = get_pretrained_wordvector(input_bf, tokenizer, bert_model, max_len = wrd_len)
    token_embeddings_bf = token_embeddings_bf.to(device) * masks_bf.unsqueeze(-1).to(device) # (atc_num_para, #wrd_len, #dim)
    # padding paragraphs
    print('1 token_embeddings_bf',token_embeddings_bf.size())
    pad_num_bf = num_para - token_embeddings_bf.size()[0]
    #print('pad_num_bf', pad_num_bf)
    if pad_num_bf>0:
        #print('>0')
        token_embeddings_bf = F.pad(input=token_embeddings_bf, pad=(0,0,0,0,0,pad_num_bf))
        print('2 token_embeddings_bf',token_embeddings_bf.size())
    elif pad_num_bf<0:
        #print('<0')
        token_embeddings_bf = token_embeddings_bf[0:60]
        print('2 token_embeddings_bf',token_embeddings_bf.size())
    else:
        token_embeddings_bf = token_embeddings_bf

    return token_embeddings, token_embeddings_bf

## define model

In [ ]:
# define Siamese model
class simple_siamese(nn.Module):
    def __init__(self, emb_dim, wrd_len, num_filters, kernel_sizes, kernel_sizes2, num_classes=2.0, dropout_rate = 0.3):
        super().__init__()
        self.emb_dim = emb_dim
        self.wrd_len = wrd_len
        self.num_filters = num_filters
        self.kernel_sizes = kernel_sizes
        self.kernel_sizes2 = kernel_sizes2
        self.dropout_rate = dropout_rate
        self.num_classes = num_classes
        


        self.conv = nn.Sequential(
            nn.Conv2d(768, 256, kernel_size = kernel_sizes), # input (#batch, 768, num_para->60, num_words->100) # kernal size = 10,50  # output: (#batch, 256, 50, 50)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(5, padding=0),  # input (#batch, 256, 50, 50) #output (#batch, 256, 10, 10)
            nn.Conv2d(256, 128,  kernel_size = kernel_sizes2), # input (#batch, 256, num_para->10, num_words->10) # kernal size = 3,3
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, padding=0),
            nn.ReLU(), 
        )

        self.fc = nn.Linear(1152, int(self.num_classes))
        self.dropout = nn.Dropout(p=dropout_rate)
        

    def forward(self, input1, input2):
        #permute input to make it fit cnn
        x1 = torch.permute(input1, (0,3,1,2))
        x2 = torch.permute(input2, (0,3,1,2))
        print(x1.size())
        print(x2.size())

        x1 = self.conv(x1)
        x2 = self.conv(x2)
        x = torch.sub(x1,x2)

        x = self.dropout(x)
        print(x.size())
        x = torch.reshape(x,(x.size()[0],-1))
        print(x.size())
        logit = self.fc(x)
        print('model output',logit.size())

        return logit

In [ ]:
emb_dim = 768
wrd_len = 100
num_filters = 128
kernel_sizes =  (10,50)
kernel_sizes2 =  (2,2)
dropout_rate = 0.3
num_classes=2.0
model = simple_siamese( emb_dim, wrd_len, num_filters, kernel_sizes, kernel_sizes2, num_classes=num_classes,dropout_rate=dropout_rate)

In [ ]:
summary(model, [(32, 60, 100, 768), (32, 60, 100, 768)])

torch.Size([32, 768, 60, 100])
torch.Size([32, 768, 60, 100])
torch.Size([32, 128, 3, 3])
torch.Size([32, 1152])
model output torch.Size([32, 2])


Layer (type:depth-idx)                   Output Shape              Param #
simple_siamese                           [32, 2]                   --
├─Sequential: 1-1                        [32, 128, 3, 3]           --
│    └─Conv2d: 2-1                       [32, 256, 51, 51]         98,304,256
│    └─ReLU: 2-2                         [32, 256, 51, 51]         --
│    └─MaxPool2d: 2-3                    [32, 256, 10, 10]         --
│    └─Conv2d: 2-4                       [32, 128, 9, 9]           131,200
│    └─ReLU: 2-5                         [32, 128, 9, 9]           --
│    └─MaxPool2d: 2-6                    [32, 128, 3, 3]           --
│    └─ReLU: 2-7                         [32, 128, 3, 3]           --
├─Sequential: 1-2                        [32, 128, 3, 3]           (recursive)
│    └─Conv2d: 2-8                       [32, 256, 51, 51]         (recursive)
│    └─ReLU: 2-9                         [32, 256, 51, 51]         --
│    └─MaxPool2d: 2-10                   [32, 256, 10,

In [ ]:
get_free_gpu()

total    : 6144
free     : 421
used     : 5722


## training and evaluation

In [ ]:
def model_eval(model, validation_dataloader, num_labels, class_weight=None):
    #tokenized_texts = []
    true_labels = []
    pred_labels = []

    threshold = 0.5

    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in validation_dataloader:
        print('val 1 free gpu',get_free_gpu())
        b_input_key = batch[0]
        b_labels = batch[1].to(device)


        #convert key to text embedding
        tk_batch = []
        tk_batch_bf = []
        #print('val batch',batch)
        for t in b_input_key.detach().to('cpu').numpy():
            tk, tk_bf = get_text_embedding(t[0], t[1], t[2], tokenizer, bert_model, para_len, wrd_len=100)
            if tk.size()[0] == 60:              
                tk_batch.append(tk)
                tk_batch_bf.append(tk_bf)
            else:
                print('token size error')
                break

        tk_batch = torch.stack(tk_batch)
        tk_batch = tk_batch.to(device)

        tk_batch_bf = torch.stack(tk_batch_bf)
        tk_batch_bf = tk_batch_bf.to(device)
        print('val 2 free gpu',get_free_gpu())

        with torch.no_grad():

            logits = model(tk_batch, tk_batch_bf)
            #loss_func = BCELoss()
            #val_loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation

            tk_batch.detach().to('cpu')
            del tk_batch
            tk_batch_bf.detach().to('cpu')
            del tk_batch_bf           
            print('val 3 free gpu',get_free_gpu())
            
            if class_weight != None:
                pos_weight = torch.tensor(class_weight).to(device)
                loss_func = BCEWithLogitsLoss(pos_weight=pos_weight)
            else:
                loss_func = BCEWithLogitsLoss()

            val_loss = loss_func(
                logits,
                b_labels.type_as(logits))  #convert labels to float for calculation

            total_eval_loss += val_loss.item()

            pred_label = torch.sigmoid(logits)
            b_labels = b_labels.to('cpu').numpy()
            pred_label = pred_label.to('cpu').numpy()

            #tokenized_texts.append(b_input_ids)
            true_labels.append(b_labels)
            pred_labels.append(pred_label)

    # Flatten outputs
    pred_labels = np.vstack(pred_labels)
    true_labels = np.vstack(true_labels)

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    return  pred_labels, true_labels, avg_val_loss

In [ ]:
def train_model(model, num_labels, train_dataloader, validation_dataloader, model_path,\
                             optimizer=None, scheduler=None, epochs = 10, \
                             class_weight = None, patience = 5):

    seed_val = 42

    threshold = 0.5
    #model_path = 'best_model.model'  # save the best model

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    training_stats = []

    best_score = -0.5
    best_epoch = 0
    cnt = 0

    total_t0 = time.time()

    if optimizer == None:
        optimizer = torch.optim.Adam(model.parameters())

    # For each epoch...
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        #print("")
        #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        #print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            
            # `batch` contains three pytorch tensors:
            #   [0]: (cik, fyear, fyear_bf)
            #   [1]: labels
            
            print('1 free gpu',get_free_gpu())
            b_input_key = batch[0]
            b_labels = batch[1].to(device)
            
            
            #convert key to text embedding
            tk_batch = []
            tk_batch_bf = []
            print('b_input_key',b_input_key)
            for t in b_input_key.detach().to('cpu').numpy():
                tk, tk_bf = get_text_embedding(t[0], t[1], t[2], tokenizer, bert_model, para_len, wrd_len=100)
                if tk.size()[0] == 60:              
                    tk_batch.append(tk)
                    tk_batch_bf.append(tk_bf)
                else:
                    print('token size error')
                    break
                
            tk_batch = torch.stack(tk_batch)
            tk_batch = tk_batch.to(device)
            
            tk_batch_bf = torch.stack(tk_batch_bf)
            tk_batch_bf = tk_batch_bf.to(device)
            print('2 free gpu',get_free_gpu())
            model.zero_grad()

            logits = model(tk_batch,tk_batch_bf)
            #print("logits shape: ", b_input_ids.size(), b_labels.size(), logits.shape())
            #loss_func = BCELoss()
            #loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation

            # add class weight
            if class_weight != None:
                pos_weight = torch.tensor(class_weight).to(device)
                loss_func = BCEWithLogitsLoss(pos_weight=pos_weight)
            else:
                loss_func = BCEWithLogitsLoss()
                
            global X 
            global Y 
            global Z
            X = logits
            Y = b_labels
            
            tk_batch.detach().to('cpu')
            del tk_batch
            tk_batch_bf.detach().to('cpu')
            del tk_batch_bf
            
            print('3 free gpu',get_free_gpu())
            print(logits.size(), b_labels.size())
#             loss = loss_func(
#                 logits.view(-1, num_labels),
#                 b_labels.type_as(logits).view(
#                     -1, num_labels))  
            # convert labels to float for calculation
            loss = loss_func(logits, b_labels.type_as(logits))
            
            Z = loss
        
        

        
            total_train_loss += loss.item()

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()

            # Update the learning rate.
            if scheduler != None:
                scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Measure how long this epoch took.
        #training_time = format_time(time.time() - t0)

        #print("")
        #print("  Average training loss: {0:.2f}".format(avg_train_loss))
        #print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        #print("")
        #print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        pred_labels, true_labels, avg_val_loss = model_eval(
            model,  validation_dataloader, num_labels, class_weight=class_weight)

        pred_bools = np.argmax(pred_labels, axis=1)
        true_bools = np.argmax(true_labels, axis=1)

        val_f1 = f1_score(true_bools, pred_bools, average=None) * 100
        val_f1 = val_f1[1]  # return f1 for  class 1
        val_acc = (
            pred_bools == true_bools).astype(int).sum() / len(pred_bools)

        #print('Validation Accuracy: {0:.4f}, F1: {1:.4f}, Loss: {2:.4f}'.format(val_f1, val_acc, avg_val_loss))
        #print(classification_report(np.array(true_labels), pred_bools, target_names=label_cols) )
        print("Epoch {0}\t Train Loss: {1:.4f}\t Val Loss {2:.4f}\t Val Acc: {3:.4f}\t Val F1: {4:.4f}".\
          format(epoch_i +1, avg_train_loss, avg_val_loss, val_acc, val_f1))

        # Measure how long the validation run took.
        #validation_time = format_time(time.time() - t0)

        #print("  Validation Loss: {0:.2f}".format(val_f1_accuracy))
        #print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append({
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': val_f1,
            'Best F1': best_score,
            'Best epoch': best_epoch
            #'Training Time': training_time,
            #'Validation Time': validation_time
        })

        # early stopping
        if val_f1 > best_score:
            best_score = val_f1
            best_epoch = epoch_i + 1
            torch.save(copy.deepcopy(model.state_dict()), model_path)
            print("model saved")
            cnt = 0
        else:
            cnt += 1
            if cnt == patience:
                print("\n")
                print("early stopping at epoch {0}".format(epoch_i + 1))

                break

    print("")
    #print("Training complete!")

    #print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    
    return model, training_stats

# training model

In [ ]:
emb_dim = 768
wrd_len = 100
para_len = 60
num_filters = 128
kernel_sizes =  (10,50)
kernel_sizes2 =  (2,2)
dropout_rate = 0.3
num_classes=2.0
batch_size = 8
para_map = para_map
class_weight = 10

result = []
label_cols = ['fraud']

df = pos_neg_pair.copy()
# create traning dataset only to test the code
df = pos_neg_pair.iloc[0:32,:]
df.loc[1,'fraud'] = 1
df.loc[30,'fraud'] = 1

In [ ]:
get_free_gpu()

total    : 6144
free     : 429
used     : 5714


In [ ]:
torch.cuda.empty_cache()

In [ ]:
get_free_gpu()

total    : 6144
free     : 3034
used     : 3109


In [ ]:
X = None
Y = None
Z = None

In [ ]:
df

,cik,fyear,count_para,fraud,fyear_bf
1,20,1996,31,1.0,1995.0
2,20,1997,34,0.0,1996.0
3,20,1998,49,0.0,1997.0
4,20,1999,40,0.0,1998.0
5,20,2000,34,0.0,1999.0
6,20,2001,38,0.0,2000.0
7,20,2002,52,0.0,2001.0
8,20,2003,61,0.0,2002.0
9,20,2004,80,0.0,2003.0
10,20,2005,71,0.0,2004.0


In [ ]:
df.shape

(32, 5)

In [ ]:
for col in label_cols:
    print("\n------------")
    print(col)
    print("------------")

    y = df[col].astype(int).values
    x_key = df[['cik', 'fyear', 'fyear_bf']].values

    fold = 0

    skf = StratifiedKFold(n_splits=2, random_state=0, shuffle=True)

    for train_index, test_index in skf.split(x_key, y):

        print("\nfold {} \n".format(fold))

        fold += 1
        X_train, X_test = x_key[train_index], x_key[test_index]
        X_train = torch.tensor(X_train)
        X_test = torch.tensor(X_test)

        Y_train, Y_test = y[train_index], y[test_index]

        Y_train = pd.get_dummies(Y_train).values
        Y_train = torch.tensor(Y_train)

        Y_test = pd.get_dummies(Y_test).values
        Y_test = torch.tensor(Y_test)

        train_dataset = TensorDataset(X_train, Y_train)
        val_dataset = TensorDataset(X_test, Y_test)

        train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler=RandomSampler(train_dataset),  # Select batches randomly
            batch_size=batch_size  # Trains with this batch size.
        )

        validation_dataloader = DataLoader(
            val_dataset,  # The validation samples.
            sampler=RandomSampler(
                val_dataset),  # Pull out batches sequentially.
            batch_size=batch_size  # Evaluate with this batch size.
        )

        if class_weight == None:
            pass
        else:
            train_sample_weight = np.array(
                [class_weight if i[1] == 1 else 1 for i in Y_train])
            test_sample_weight = np.array(
                [class_weight if i[1] == 1 else 1 for i in Y_test])

        model_name = "./model/simple_siamese_" + str(fold)
        #model = cnn(emb_dim, seq_len, num_filters, kernel_sizes, num_labels)
        model = simple_siamese(emb_dim, wrd_len, num_filters, kernel_sizes,\
                               kernel_sizes2, num_classes=num_classes, dropout_rate = dropout_rate)
        model.to(device)


        model, training_stats = train_model(model, num_classes, train_dataloader, validation_dataloader, \
                                                        model_path = model_name, class_weight = class_weight,\
                                                        optimizer=None, scheduler=None, epochs = 20)

        print("load the best model ... ")

        model.load_state_dict(torch.load(model_name))

        # show performance of best model
        model.eval()
        pred_labels, true_labels,avg_val_loss = model_eval(model, \
                                                validation_dataloader, num_classes, class_weight = class_weight)

        pred_bools = np.argmax(pred_labels, axis = 1)
        true_bools = np.argmax(true_labels, axis = 1)

        p, r, f, _ = precision_recall_fscore_support(true_bools,pred_bools, pos_label = 1)
        #val_f1 = f1_score(true_bools,pred_bools, average = None)*100
        #val_f1 = val_f1[1] # return f1 for  class 1
        val_acc = (pred_bools == true_bools).astype(int).sum()/len(pred_bools)

        print('Precision: {0:.4f}, Recall: {1:.4f}, F1: {2:.4f}, Loss: {3:.4f}'.format(p[1], r[1], f[1], avg_val_loss))
        print(classification_report(true_bools, pred_bools) )


        result.append([col, fold, p[1], r[1], f[1], training_stats[-1]["Best epoch"]])
        
        torch.cuda.empty_cache()
        get_free_gpu()
        


------------
fraud
------------

fold 0 



simple_siamese(
  (conv): Sequential(
    (0): Conv2d(768, 256, kernel_size=(10, 50), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(256, 128, kernel_size=(2, 2), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU()
  )
  (fc): Linear(in_features=1152, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


total    : 6144
free     : 2665
used     : 3478
1 free gpu None
b_input_key tensor([[  20., 1996., 1995.],
        [  20., 2005., 2004.],
        [  20., 2008., 2007.],
        [1750., 2002., 2001.],
        [  20., 2002., 2001.],
        [1750., 2003., 2002.],
        [1750., 1996., 1995.],
        [  20., 2006., 2005.]], dtype=torch.float64)
20.0 1996.0 1995.0
1 token_embeddings torch.Size([31, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([41, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 2005.0 2004.0
1 token_embeddings torch.Size([71, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([80, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 2008.0 2007.0
1 token_embeddings torch.Size([81, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([79, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
1750.0 2002.0 2001.0
1 toke

1 token_embeddings_bf torch.Size([31, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
total    : 6144
free     : 30
used     : 6113
val 2 free gpu None
torch.Size([8, 768, 60, 100])
torch.Size([8, 768, 60, 100])
torch.Size([8, 128, 3, 3])
torch.Size([8, 1152])
model output torch.Size([8, 2])
total    : 6144
free     : 25
used     : 6118
val 3 free gpu None
Epoch 1	 Train Loss: 15.2675	 Val Loss 40.8636	 Val Acc: 0.3125	 Val F1: 15.3846
model saved
total    : 6144
free     : 19
used     : 6124
1 free gpu None
b_input_key tensor([[  20., 2003., 2002.],
        [1750., 2003., 2002.],
        [  20., 1996., 1995.],
        [  20., 2001., 2000.],
        [  20., 2006., 2005.],
        [1750., 2011., 2010.],
        [  20., 2002., 2001.],
        [1750., 2012., 2011.]], dtype=torch.float64)
20.0 2003.0 2002.0
1 token_embeddings torch.Size([61, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([52, 100, 768])
2 token_embeddings_bf torch.Size

...

total    : 6144
free     : 59
used     : 6084
val 3 free gpu None
total    : 6144
free     : 59
used     : 6084
val 1 free gpu None
20.0 2003.0 2002.0
1 token_embeddings torch.Size([61, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([52, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 2005.0 2004.0
1 token_embeddings torch.Size([71, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([80, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
1750.0 2007.0 2006.0
1 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([59, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 1996.0 1995.0
1 token_embeddings torch.Size([31, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([41, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 2002.0 2001.0
1 token_embeddings torch.Size([52, 100, 768])
2 token_emb

1 token_embeddings torch.Size([61, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([52, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
total    : 6144
free     : 48
used     : 6095
val 2 free gpu None
torch.Size([8, 768, 60, 100])
torch.Size([8, 768, 60, 100])
torch.Size([8, 128, 3, 3])
torch.Size([8, 1152])
model output torch.Size([8, 2])
total    : 6144
free     : 60
used     : 6083
val 3 free gpu None
total    : 6144
free     : 60
used     : 6083
val 1 free gpu None
20.0 1998.0 1997.0
1 token_embeddings torch.Size([49, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([34, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
1750.0 1996.0 1995.0
1 token_embeddings torch.Size([25, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([23, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 2006.0 2005.0
1 token_embeddings torch.Size([7

<All keys matched successfully>

simple_siamese(
  (conv): Sequential(
    (0): Conv2d(768, 256, kernel_size=(10, 50), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(256, 128, kernel_size=(2, 2), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU()
  )
  (fc): Linear(in_features=1152, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

total    : 6144
free     : 52
used     : 6091
val 1 free gpu None
1750.0 2011.0 2010.0
1 token_embeddings torch.Size([76, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([57, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 1998.0 1997.0
1 token_embeddings torch.Size([49, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([34, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
1750.0 2012.0 2011.0
1 token_embeddings torch.Size([61, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([76, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 2003.0 2002.0
1 token_embeddings torch.Size([61, 100, 768])
2 token_embeddings torch.Size([60, 100, 768])
1 token_embeddings_bf torch.Size([52, 100, 768])
2 token_embeddings_bf torch.Size([60, 100, 768])
20.0 1996.0 1995.0
1 token_embeddings torch.Size([31, 100, 768])
2 token_embeddings torch.Size

In [ ]:
result

[['fraud', 1, 0.08333333333333333, 1.0, 0.15384615384615385, 1],
 ['fraud', 2, 0.07142857142857142, 1.0, 0.13333333333333333, 1]]